# Mini-Proj 2 

### Group Members: Fernando Garcia, Gavin Wright, Chetas Patel ,Zhenyang Ding , Kevin Lee

### Due: Nov 17

In [30]:
#imports
import numpy as np
import pandas as pd
import os

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

## Datasets

In [31]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

# 20newsgroups Data set
twenty_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes')) # bunch object


# IMDB Data set
def load_reviews(data_dir):
    pos_dir = os.path.join(data_dir, 'pos')
    neg_dir = os.path.join(data_dir, 'neg')

    train_reviews = []
    train_labels = []
    test_reviews = []
    test_labels = []

    
    def load_files(directory, label):
        tt = 0
        for filename in os.listdir(directory):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                tt+=1
                if(tt%10 < 3):
                    train_reviews.append(file.read())
                    train_labels.append(label)
                else:
                    test_reviews.append(file.read())
                    test_labels.append(label)

    load_files(pos_dir, 1)
    load_files(neg_dir, 0)

    return train_reviews, train_labels, test_reviews, test_labels

train_data_dir = 'aclImdb/train/'
train_reviews, train_labels, test_reviews, test_labels = load_reviews(train_data_dir)

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_reviews)
X_train_counts.shape
count_vect.vocabulary_.get(u'algorithm')


In [33]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(7500, 45901)

In [34]:
LRclf = LogisticRegression(random_state=0).fit(X_train_tfidf, train_labels)
DTclf = DecisionTreeClassifier(random_state=0).fit(X_train_tfidf, train_labels)
SVMclf = make_pipeline(StandardScaler(),LinearSVC(dual="auto", random_state=0, tol=1e-5))
ABclf = AdaBoostClassifier(n_estimators=100, random_state=0)
RFclf =RandomForestClassifier(max_depth=2, random_state=0)


In [35]:
from sklearn.pipeline import Pipeline
LRtext_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('LRclf', LogisticRegression()),
])
DTtext_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('DTclf', DecisionTreeClassifier()),
])
SVMtext_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('SVMclf', LinearSVC()),
])
ABtext_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('ABclf', AdaBoostClassifier()),
])
RFtext_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('RFclf', RandomForestClassifier()),
])

## LogReg

In [36]:
LRtext_clf.fit(train_reviews, train_labels)
docs_test = train_reviews
predicted = LRtext_clf.predict(docs_test)
np.mean(predicted == train_labels)

0.9402666666666667

## Decision tree


In [37]:
torture=cross_val_score(DTtext_clf,test_reviews,test_labels , cv=10)
torture.mean()

0.6904000000000001

## SVM

In [38]:
SVMtext_clf.fit(train_reviews, train_labels)
docs_test = test_reviews
predicted = SVMtext_clf.predict(docs_test)
np.mean(predicted == test_labels)

c:\Users\ferna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


0.8782857142857143

## AdaBoost

In [40]:
ABtext_clf.fit(train_reviews,train_labels)
docs_test = test_reviews
predicted = ABtext_clf.predict(docs_test)
ABtext_clf.score(test_reviews,test_labels )

KeyboardInterrupt: 

## Random Forest

In [ ]:
RFtext_clf.fit(train_reviews, train_labels)
docs_test = test_reviews
predicted = RFtext_clf.predict(docs_test)
np.mean(predicted == test_labels)

1.0